In [ ]:
import sympy as sp
x,y,z = sp.symbols('x y z')
def grad(f):
    Gf = sp.Matrix([[f.diff(x),f.diff(y),f.diff(z)]])
    return Gf
def curl(f):
    Cf = sp.Matrix([[f[2].diff(y)-f[1].diff(z),f[0].diff(z)-f[2].diff(x),f[1].diff(x)-f[0].diff(y)]])
    return Cf
def div(f):
    return sp.Matrix([[f[0].diff(x)+f[1].diff(y)+f[2].diff(z)]])

def ev(f,ax,ay,az):
    return f.subs(x,ax).subs(y,ay).subs(z,az)


In [ ]:
Basis0 = [
    sp.Matrix([[1]])
]
Basis1 = [
    sp.Matrix([[1,0,0]]),
    sp.Matrix([[0,1,0]]),
    sp.Matrix([[0,0,1]])
]
Basis2 = [
    sp.Matrix([[1,0,0]]),
    sp.Matrix([[0,1,0]]),
    sp.Matrix([[0,0,1]])
]
Basis3 = [
    sp.Matrix([[1]])
]

In [ ]:
def printEigenRow(A):
    print("{",sp.ccode(A[0]),sep='',end='')
    for i in range(1,A.cols):
        print(",",sp.ccode(A[i]),sep='',end='')
    print("}",sep='',end='')
def printEigen(A):
    if (A.cols==1 or A.rows==1):
        printEigenRow(A)
    else:
        print("{{",sep='',end='')
        printEigenRow(A.row(0))
        for i in range(1,A.rows):
            print(",",sep='',end='')
            printEigenRow(A.row(i))
        print("}}",sep='',end='')
def printVSpaces(B):
    print("    const std::array<VType,Dim> VSpaces{{")
    for i in range(len(B)):
        if (i > 0):
            print(",")
        print("      VType",end='')
        printEigen(B[i])
    print("\n    }};")
def printMass(M):
    print("    const Eigen::Matrix<double,Dim,Dim> mass",end='')
    printEigen(M)
    print(";")
def printDiff(B):
    print("    const std::array<MDiff,3> diff{{")
    for i in range(len(B)):
        if (i > 0):
            print(",")
        print("      MDiff",end='')
        printEigen(B[i])
    print("\n    }};")
f = sp.Function('f')(x,y,z)
def mass(B):
    M = sp.Matrix.zeros(len(B),len(B))
    for i in range(len(B)):
        for j in range(len(B)):
            M[i,j] = (B[i]*B[j].transpose()).trace()
    return M
def decompBasisOrth(B,Bp,op):
    dx = sp.Matrix.zeros(len(Bp),len(B))
    dy = sp.Matrix.zeros(len(Bp),len(B))
    dz = sp.Matrix.zeros(len(Bp),len(B))
    for i in range(len(B)):
        diff = op(f*B[i])
        for j in range(len(Bp)):
            extract = (sp.hadamard_product(diff,Bp[j])*sp.Matrix.ones(diff.cols,diff.rows)).trace()
            if (extract.has(f.diff(x))):
                dx[j,i] = extract/f.diff(x)
            if (extract.has(f.diff(y))):
                dy[j,i] = extract/f.diff(y)
            if (extract.has(f.diff(z))):
                dz[j,i] = extract/f.diff(z)
    return [dx,dy,dz]
def genBasis(B,Bp,op):
    massB = mass(B)
    massBp = mass(Bp)
    P = massBp.diagonalize(normalize=True)[0]
    BpP = [Bp[0]*0]*P.rows
    for i in range(P.rows):
        for j in range(P.cols):
            BpP[i] += P[i,j]*Bp[j]
    D = mass(BpP)
    dx,dy,dz = decompBasisOrth(B,BpP,op)
    for i in range(P.rows):
        for j in range(len(B)):
            dx[i,j] /= D[i,i]
            dy[i,j] /= D[i,i]
            dz[i,j] /= D[i,i]
    dx2 = P.transpose()*dx
    dy2 = P.transpose()*dy
    dz2 = P.transpose()*dz
    printVSpaces(B)
    printMass(massB)
    printDiff([dx2,dy2,dz2])

In [ ]:
genBasis(Basis0,Basis1,grad)

In [ ]:
genBasis(Basis1,Basis2,curl)

In [ ]:
genBasis(Basis2,Basis3,div)